# Object Volume Measurement using ArUco Marker

This notebook demonstrates a **hybrid computer vision approach** to measure the **real-world dimensions and estimated volume** of an object in a video or live camera feed.  
The method combines **Aruco-based scale calibration** with **image-based contour analysis**, producing a robust, practical system for general object measurement and expansion tracking.

## Import Libraries

In [1]:
import cv2
import numpy as np
import os
import csv
import time
from datetime import datetime

## Initialize ArUco Detector

We use OpenCV’s built-in ArUco module to detect a specific marker pattern (e.g., 4x4 dictionary).  
This provides a known physical reference to convert pixel measurements into centimeters.

In [2]:
def setup_aruco_detector():
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    aruco_params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)
    return detector

## Detect ArUco Marker and Compute Scale

This function returns:
- `cm_per_pixel`: conversion factor between pixels and centimeters  
- `corners`: marker corner coordinates for visualization  
- `measured_marker_cm`: pixel-based marker size (for reference)


In [3]:
def detect_aruco_scale(gray, detector, marker_id, marker_size_cm):
    """Detect ArUco marker and compute cm/pixel scaling factor."""
    corners, ids, _ = detector.detectMarkers(gray)
    if ids is not None and marker_id in ids:
        idx = np.where(ids == marker_id)[0][0]
        c = corners[idx][0]
        side_px = np.mean([
            np.linalg.norm(c[0] - c[1]),
            np.linalg.norm(c[1] - c[2]),
            np.linalg.norm(c[2] - c[3]),
            np.linalg.norm(c[3] - c[0]),
        ])
        cm_per_pixel = marker_size_cm / side_px
        return cm_per_pixel, c
    return None, None

## Image Preprocessing and Object Detection

Enhances visibility and contrast for stable object contour extraction.

In [4]:
class DetectorObj:
    def __init__(self):
        pass

    def detect_objects(self, frame):
        """Detects large objects using adaptive thresholding and contour extraction."""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mask = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY_INV, 19, 5
        )
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return [cnt for cnt in contours if cv2.contourArea(cnt) > 2000]

## Compute Volume and Error Margin
Uses Bai et al. (2006) volume estimation and calculates weighted physical error.

In [5]:
def compute_volume_and_error(contour, cm_per_pixel, true_dims=(17.5, 6.5)):
    """Compute width, height, volume, and margin of error."""
    x, y, w, h = cv2.boundingRect(contour)
    bun_mask = np.zeros((h, w), np.uint8)
    cv2.drawContours(bun_mask, [contour - [x, y]], -1, 255, -1)

    diameters_px = []
    for col in range(w):
        col_pix = np.where(bun_mask[:, col] > 0)[0]
        if len(col_pix) > 1:
            diameters_px.append(col_pix.max() - col_pix.min())
    if len(diameters_px) <= 10:
        return None

    diameters_px = np.array(diameters_px)
    diameters_cm = diameters_px * cm_per_pixel
    delta_x_cm = cm_per_pixel
    volume_cm3 = np.sum(np.pi * (diameters_cm / 2) ** 2 * delta_x_cm)

    w_cm = w * cm_per_pixel
    h_cm = h * cm_per_pixel

    true_w, true_h = true_dims
    width_diff_pct = abs((w_cm - true_w) / true_w) * 10
    height_diff_pct = abs((h_cm - true_h) / true_h) * 10
    err_total = np.sqrt(0.7 * (width_diff_pct ** 2) + 0.3 * (height_diff_pct ** 2))

    return w_cm, h_cm, volume_cm3, err_total, (x, y, w, h)

## CSV Logger Helpers
Initialize and append to CSV file for measurement tracking.

In [6]:
_last_log_time = 0.0
LOG_INTERVAL_SEC = 1.0


def init_csv_logger():
    os.makedirs("data", exist_ok=True)
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"data/Object_Measurements_{timestamp}.csv"
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Timestamp", "Frame Index", "Object Index", "Marker Status",
            "cm/pixel", "Width (cm)", "Height (cm)", "Volume (cm^3)", "Margin of Error (%)"
        ])
    return filename


def log_measurement(filename, frame_idx, object_idx, marker_status,
                    cm_per_pixel, w_cm, h_cm, vol_cm3, err_total):
    global _last_log_time
    current_time = time.time()
    if current_time - _last_log_time < LOG_INTERVAL_SEC:
        return
    _last_log_time = current_time

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            timestamp, frame_idx, object_idx, marker_status,
            round(cm_per_pixel, 6), round(w_cm, 3),
            round(h_cm, 3), round(vol_cm3, 3), round(err_total, 3)
        ])

## Main Measurement Loop
Integrates all modules:
- Captures frames
- Detects ArUco marker
- Processes object
- Computes measurements
- Logs to CSV

In [7]:
def run_measurement(use_webcam=True, video_path="assets/video2.mp4", camera_index=0):
    """Unified runner for image, video, or webcam with CSV logging."""
    if not use_webcam and os.path.isfile(video_path):
        ext = os.path.splitext(video_path)[1].lower()
        mode = "image" if ext in [".jpg", ".jpeg", ".png", ".bmp"] else "video"
    else:
        mode = "camera"

    aruco_detector = setup_aruco_detector()
    detector_obj = DetectorObj()
    csv_filename = init_csv_logger()

    marker_id_local, marker_size_cm_local = 0, 5.0
    last_cm_per_pixel, soft_mode = None, True

    print(f"▶️ Starting measurement in {mode.upper()} mode... Press 'q' to quit.")

    # --- IMAGE MODE ---
    if mode == "image":
        os.makedirs("data/results", exist_ok=True)
        frame = cv2.imread(video_path)
        if frame is None:
            raise IOError(f"Cannot read image: {video_path}")
        vis = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        cm_per_pixel, marker_corners = detect_aruco_scale(gray, aruco_detector, marker_id_local, marker_size_cm_local)
        if marker_corners is not None:
            cv2.polylines(vis, [np.intp(marker_corners)], True, (255, 0, 0), 2)
            last_cm_per_pixel = cm_per_pixel
            marker_status = f"Marker OK (ID {marker_id_local})"
        else:
            cm_per_pixel = last_cm_per_pixel or 0.05  # fallback
            marker_status = "Marker not found - using last/fallback scale"

        contours = detector_obj.detect_objects(vis)
        for obj_idx, cnt in enumerate(contours, start=1):
            result = compute_volume_and_error(cnt, cm_per_pixel)
            if not result:
                continue
            w_cm, h_cm, vol_cm3, err_total, (x, y, w, h) = result
            box = np.intp(cv2.boxPoints(cv2.minAreaRect(cnt)))
            cv2.polylines(vis, [box], True, (0, 255, 0), 2)
            cv2.putText(vis, f"W: {w_cm:.2f}cm H: {h_cm:.2f}cm", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(vis, f"Vol: {vol_cm3:.1f}cm³ +/-{err_total:.1f}%", (x, y + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            log_measurement(csv_filename, 1, obj_idx, marker_status,
                            cm_per_pixel, w_cm, h_cm, vol_cm3, err_total)

        result_path = os.path.join("data/results", f"result_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.jpg")
        cv2.imwrite(result_path, vis)
        cv2.imshow("Measurement (Photo)", vis)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        print(f"✅ Saved result image: {result_path}")
        print(f"✅ CSV logged: {csv_filename}")
        return

    # --- VIDEO OR CAMERA MODE ---
    cap = cv2.VideoCapture(camera_index if mode == "camera" else video_path)
    if not cap.isOpened():
        raise IOError(f"❌ Cannot open camera index {camera_index}")

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        vis = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        cm_per_pixel, marker_corners = detect_aruco_scale(gray, aruco_detector, marker_id_local, marker_size_cm_local)
        if marker_corners is not None:
            cv2.polylines(vis, [np.intp(marker_corners)], True, (255, 0, 0), 2)
            last_cm_per_pixel = cm_per_pixel
            marker_status = f"Marker OK (ID {marker_id_local})"
        else:
            cm_per_pixel = last_cm_per_pixel or 0.05
            marker_status = "Marker lost - using last/fallback scale"

        contours = detector_obj.detect_objects(vis)
        for obj_idx, cnt in enumerate(contours, start=1):
            result = compute_volume_and_error(cnt, cm_per_pixel)
            if not result:
                continue
            w_cm, h_cm, vol_cm3, err_total, (x, y, w, h) = result
            box = np.intp(cv2.boxPoints(cv2.minAreaRect(cnt)))
            cv2.polylines(vis, [box], True, (0, 255, 0), 2)
            cv2.putText(vis, f"Width:{w_cm:.2f}cm Height:{h_cm:.2f}cm", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(vis, f"Vol:{vol_cm3:.1f}cm^3 +/-{err_total:.1f}%", (x, y + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            log_measurement(csv_filename, frame_idx, obj_idx, marker_status,
                            cm_per_pixel, w_cm, h_cm, vol_cm3, err_total)

        cv2.putText(vis, marker_status, (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 200, 255), 2)
        cv2.imshow("Measurement (Live)", vis)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"✅ CSV logged to: {csv_filename}")


In [11]:
if __name__ == "__main__":
    # 👇 Change camera_index to select different webcams
    run_measurement(use_webcam=False, video_path="assets/video4.mp4", camera_index=1)  # 0 = built-in, 1 = external

▶️ Starting measurement in VIDEO mode... Press 'q' to quit.
✅ CSV logged to: data/Object_Measurements_2025-11-11_13-32-10.csv
